In [1]:
from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

import numpy as np 
import pandas as pd 
import io
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.array(x_train, np.float32)
x_test = np.array(x_test, np.float32)

num_features=784
x_train = x_train.reshape(60000, num_features)
x_test = x_test.reshape(10000, num_features)

x_train = x_train / 255
x_test = x_test /255

x_train

11501568/11490434 [==============================] - 0s 0us/step


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [3]:
x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [4]:
x_train.shape

(60000, 784)

In [5]:
x_test.shape

(10000, 784)

In [6]:
y_train.shape

(60000,)

In [7]:
num_classes = 10

num_features = 784

learning_rate = 0.01

training_steps = 1000

batch_size = 256

display_step = 50

In [8]:
num_batches = int(x_train.shape[0] / batch_size)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [9]:
W = tf.Variable(np.random.randn(784, 10).astype(np.float32))

B = tf.Variable(np.random.randn(10).astype(np.float32))


In [10]:
def logistic_regression(x):
    return tf.nn.softmax(tf.add(tf.matmul(x, W), B))
    
def cross_entropy(y_pred, y_true):

    y_true = tf.one_hot(y_true, depth = num_classes)

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)    

    loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    return loss
    

In [11]:
def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
optimizer = tf.optimizers.SGD(learning_rate)

def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    gradients = g.gradient(loss, [W, B])
    optimizer.apply_gradients(zip(gradients, [W, B]))


In [13]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    run_optimization(batch_x, batch_y)
    if step % display_step == 0:

        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print(f"step: {step}, loss: {loss}, accuracy: {acc}")


step: 50, loss: 326.7994384765625, accuracy: 0.7734375
step: 100, loss: 141.368408203125, accuracy: 0.890625
step: 150, loss: 190.04734802246094, accuracy: 0.8203125
step: 200, loss: 106.4253158569336, accuracy: 0.87890625
step: 250, loss: 117.43205261230469, accuracy: 0.890625
step: 300, loss: 140.68804931640625, accuracy: 0.91796875
step: 350, loss: 263.34991455078125, accuracy: 0.83984375
step: 400, loss: 81.66917419433594, accuracy: 0.8984375
step: 450, loss: 75.05609130859375, accuracy: 0.93359375
step: 500, loss: 249.3470458984375, accuracy: 0.828125
step: 550, loss: 258.92767333984375, accuracy: 0.8046875
step: 600, loss: 81.95301818847656, accuracy: 0.91015625
step: 650, loss: 107.12532043457031, accuracy: 0.91796875
step: 700, loss: 73.60589599609375, accuracy: 0.9296875
step: 750, loss: 125.22872161865234, accuracy: 0.9140625
step: 800, loss: 84.41012573242188, accuracy: 0.921875
step: 850, loss: 93.07411193847656, accuracy: 0.921875
step: 900, loss: 87.3175277709961, accurac

In [14]:
pred = logistic_regression(x_test)
a = accuracy(pred, y_test)
print(f"Accuracy: {a}")

Accuracy: 0.9032999873161316
